## Building a Tree

First we need to define the TreeNode Class to start building a tree

In [2]:
class BSTNode:
    def __init__(self, data=None, left=None, right=None):
        self.data = data
        self.left = left
        self.right = right

    def printTree(self):
        printBTree(self,lambda n:(str(n.data),n.left,n.right))

import functools as fn
def printBTree(node, nodeInfo=None, inverted=False, isTop=True):

   # node value string and sub nodes
   stringValue, leftNode, rightNode = nodeInfo(node)

   stringValueWidth  = len(stringValue)

   # recurse to sub nodes to obtain line blocks on left and right
   leftTextBlock     = [] if not leftNode else printBTree(leftNode,nodeInfo,inverted,False)

   rightTextBlock    = [] if not rightNode else printBTree(rightNode,nodeInfo,inverted,False)

   # count common and maximum number of sub node lines
   commonLines       = min(len(leftTextBlock),len(rightTextBlock))
   subLevelLines     = max(len(rightTextBlock),len(leftTextBlock))

   # extend lines on shallower side to get same number of lines on both sides
   leftSubLines      = leftTextBlock  + [""] *  (subLevelLines - len(leftTextBlock))
   rightSubLines     = rightTextBlock + [""] *  (subLevelLines - len(rightTextBlock))

   # compute location of value or link bar for all left and right sub nodes
   #   * left node's value ends at line's width
   #   * right node's value starts after initial spaces
   leftLineWidths    = [ len(line) for line in leftSubLines  ]                            
   rightLineIndents  = [ len(line)-len(line.lstrip(" ")) for line in rightSubLines ]

   # top line value locations, will be used to determine position of current node & link bars
   firstLeftWidth    = (leftLineWidths   + [0])[0]  
   firstRightIndent  = (rightLineIndents + [0])[0] 

   # width of sub node link under node value (i.e. with slashes if any)
   # aims to center link bars under the value if value is wide enough
   # 
   # ValueLine:    v     vv    vvvvvv   vvvvv
   # LinkLine:    / \   /  \    /  \     / \ 
   #
   linkSpacing       = min(stringValueWidth, 2 - stringValueWidth % 2)
   leftLinkBar       = 1 if leftNode  else 0
   rightLinkBar      = 1 if rightNode else 0
   minLinkWidth      = leftLinkBar + linkSpacing + rightLinkBar
   valueOffset       = (stringValueWidth - linkSpacing) // 2

   # find optimal position for right side top node
   #   * must allow room for link bars above and between left and right top nodes
   #   * must not overlap lower level nodes on any given line (allow gap of minSpacing)
   #   * can be offset to the left if lower subNodes of right node 
   #     have no overlap with subNodes of left node                                                                                                                                 
   minSpacing        = 2
   rightNodePosition = fn.reduce(lambda r,i: max(r,i[0] + minSpacing + firstRightIndent - i[1]), \
                                 zip(leftLineWidths,rightLineIndents[0:commonLines]), \
                                 firstLeftWidth + minLinkWidth)

   # extend basic link bars (slashes) with underlines to reach left and right
   # top nodes.  
   #
   #        vvvvv
   #       __/ \__
   #      L       R
   #
   linkExtraWidth    = max(0, rightNodePosition - firstLeftWidth - minLinkWidth )
   rightLinkExtra    = linkExtraWidth // 2
   leftLinkExtra     = linkExtraWidth - rightLinkExtra

   # build value line taking into account left indent and link bar extension (on left side)
   valueIndent       = max(0, firstLeftWidth + leftLinkExtra + leftLinkBar - valueOffset)
   valueLine         = " " * max(0,valueIndent) + stringValue
   slash             = "\\" if inverted else  "/"
   backslash         = "/" if inverted else  "\\"
   uLine             = "¯" if inverted else  "_"

   # build left side of link line
   leftLink          = "" if not leftNode else ( " " * firstLeftWidth + uLine * leftLinkExtra + slash)

   # build right side of link line (includes blank spaces under top node value) 
   rightLinkOffset   = linkSpacing + valueOffset * (1 - leftLinkBar)                      
   rightLink         = "" if not rightNode else ( " " * rightLinkOffset + backslash + uLine * rightLinkExtra )

   # full link line (will be empty if there are no sub nodes)                                                                                                    
   linkLine          = leftLink + rightLink

   # will need to offset left side lines if right side sub nodes extend beyond left margin
   # can happen if left subtree is shorter (in height) than right side subtree                                                
   leftIndentWidth   = max(0,firstRightIndent - rightNodePosition) 
   leftIndent        = " " * leftIndentWidth
   indentedLeftLines = [ (leftIndent if line else "") + line for line in leftSubLines ]

   # compute distance between left and right sublines based on their value position
   # can be negative if leading spaces need to be removed from right side
   mergeOffsets      = [ len(line) for line in indentedLeftLines ]
   mergeOffsets      = [ leftIndentWidth + rightNodePosition - firstRightIndent - w for w in mergeOffsets ]
   mergeOffsets      = [ p if rightSubLines[i] else 0 for i,p in enumerate(mergeOffsets) ]

   # combine left and right lines using computed offsets
   #   * indented left sub lines
   #   * spaces between left and right lines
   #   * right sub line with extra leading blanks removed.
   mergedSubLines    = zip(range(len(mergeOffsets)), mergeOffsets, indentedLeftLines)
   mergedSubLines    = [ (i,p,line + (" " * max(0,p)) )       for i,p,line in mergedSubLines ]
   mergedSubLines    = [ line + rightSubLines[i][max(0,-p):]  for i,p,line in mergedSubLines ]                        

   # Assemble final result combining
   #  * node value string
   #  * link line (if any)
   #  * merged lines from left and right sub trees (if any)
   treeLines = [leftIndent + valueLine] + ( [] if not linkLine else [leftIndent + linkLine] ) + mergedSubLines

   # invert final result if requested
   treeLines = reversed(treeLines) if inverted and isTop else treeLines

   # return intermediate tree lines or print final result
   if isTop : print("\n".join(treeLines))
   else     : return treeLines      

Next let's start building a tree, a simple one that looks like this:

```
    5
   / \
  3   6
 / \   \
2   4   7

```

In [3]:
# This will build the above tree
leaf1 = BSTNode(2)
leaf2 = BSTNode(4)
leaf3 = BSTNode(7)

node3 = BSTNode(3, leaf1, leaf2)
node6 = BSTNode(6, None, leaf3)

root = BSTNode(5, node3, node6)

root.printTree()

    5
   / \
  3   6
 / \   \
2   4   7


# Traversing a BST

To search for a node in a BST, we can take advantage of the required ordering of the BST.

In [4]:
def searchBST(root, val):
    if not root:
        return None
    if root.data == val:
        return root
    if val < root.data:
        return searchBST(root.left, val)
    if val > root.data:
        return searchBST(root.right, val)

In [5]:
# Searching for the node with value 3
foundNode = searchBST(root, 3)
foundNode.data

3

## Deleting a Node in a BST

Deletion of a node can get quite complicated because you have to update the below nodes in the absence of the deleted node.

Take this tree example below:

```        
       10
      /   
     5    
    / \
   3   8
  / \ / \
 2  1 7 9
```


This is the most complex scenario. 

If we had to remove the node with value 5, we need to shift the below subtrees since there's so much children attached. 

Here's a valid subtree post-deletion:

```        
       10
      /   
     7    
    / \
   3   8
  / \   \
 2  1    9         
```

In order to preserve the ordering, we must go to the right subtree, and then go all the way left. This will essentially replace the value with left-most leaf node of the right subtree of the to-be deleted node. 

The steps are 

- Replace the value with left most leaf node in the right subtree
- Delete the left most leaf node in the right subtree

We want to do it this way as deleting a leaf node is much cleaner than trying to move pointers in this complex scenario

Other scenarios such as deleting a leaf node are much simpler, so here's the code to handle them

In [6]:
def deleteNode(self, root, key):
        # This is mostly the same as the searching algorithm with a few key differences!
        if not root:
            return None
        
        if root.val == key:
            # If there's no subtree to the right, just return the left subtree as the next node to the parent
            if not root.right:
                return root.left
            # Vice versa for the left subtree being empty
            if not root.left:
                return root.right
            
            # If both exist, we come to this code block
            # We need to go right first
            if root.left and root.right:
                tmp = root.right
                while tmp.left:
                    # Go all the way to the left of the tree
                    tmp = tmp.left
                # Update the root with that value, so that we can delete that node with the same value further down the right subtree
                root.val = tmp.val    
                # Update the node to be to deleted as the the value, recursion will delete it with out preexisting logic
                root.right = self.deleteNode(root.right, root.val)
            
        elif key < root.val:
            # set the parent to left.. GENIUS MOVE as this takes care of the parent connection handling
            root.left = self.deleteNode(root.left, key)
            
        elif key > root.val:
            # set the parent to whatever's on the right
            root.right = self.deleteNode(root.right, key)
        
        return root

## Insertion into a BST

Insertion into a BST is also similar to search and deletion.

It's as simple as search, but we assign root left and root right sections with each recursive pass

At the end when we reach the end of a BST and the result is null, we return the new TreeNode object with the value

In [7]:
def insertIntoBST(self, root, val):    
    if not root:
        return TreeNode(val)
    
    if val < root.val:
        root.left = self.insertIntoBST(root.left, val)
    elif val > root.val:
        root.right = self.insertIntoBST(root.right, val)
        
    return root

### Iterative Insertion Solution

Here's also a decent iterative insertion into a BST

It's pretty easy since we just go left and right until we hit a null node. Using a parent node along the way allows for attaching the new node to it

In [8]:
def insertIntoBSTIterative(self, root, val):    

    new_node = TreeNode(val)
    
    if not root:
        return new_node
    
    tree_iter = root
    
    while tree_iter:
        parent_node = tree_iter
        if val < tree_iter.val:
            tree_iter = tree_iter.left
        elif val > tree_iter.val:
            tree_iter = tree_iter.right
    if val < parent_node.val:
        parent_node.left = new_node
    else:
        parent_node.right = new_node
        
    return root